In [2]:
from Bio import SearchIO
from Bio.SeqIO import *

from tst.domutil.util import *
from tst.utils_db import *


# os.environ["SEQlib"] = full("$DATA/sequence-data")

In [3]:
import sys
reload(sys.modules["tst.domutil.util"])
from tst.domutil.util import *

In [4]:
# reload(tst.domutil.util)
%load_ext autoreload
%autoreload 0

In [5]:
### Manually generated tmp.hmmpf and tmp.domtbl

### search_
wkdir = "/data/sequence-data/"
hmm_profile = "%s/tmp.hmmpf" % wkdir
hmm_domtbl = "%s/tmp.domtbl" % wkdir
# CATH_S35

# kwargs = {"format":"hmmer3-domtab"}
tbl_format = "hmmsearch3-domtab"

In [ ]:
import os
os.environ["SEQlib"]

## import sequence database

In [ ]:
print seqDB_curr.sequence_set.count()
print sequence.objects.count()

In [13]:
jdict = {
    'name':'CATH',
    'version':'4_1_0'}
seqDB_curr = verify_exist_entry(jdict,seqDB)

INPUT = full("$SEQlib/cath-domain-seqs-S100.fa")
fmt = 'fasta'
p_cathFAheader=re.compile('\|([0-9,a-z,A-Z]*)\/')
f_header = lambda s:p_cathFAheader.findall(s.id)[0]


In [6]:
jdict = {
    'name':'CATH-S40-nr',
    'version':'4_1_0'}
seqDB_curr = verify_exist_entry(jdict,seqDB)

INPUT = full("$SEQlib/cath-dataset-nonredundant-S40.fa")
fmt = 'fasta'
p_cathFAheader=re.compile('\|([0-9,a-z,A-Z]*)\/')
header2acc = lambda s:p_cathFAheader.findall(s.id)[0]


In [15]:
jdict = {
    'name':'Pfam-A',
    'version':'rp15'}
seqDB_curr = verify_exist_entry(jdict,seqDB)


INPUT = full("$SEQlib/Pfam-A_rp15")
fmt = 'stockholm'
header2acc = lambda s: s.name


In [17]:
%%time
def import_seqDB():
    behave = "import sequence database"

    if "debug" not in locals(): debug = 0
    reset_database_connection()
    from Bio import SeqIO

    p_clean = re.compile('[^-,^\.]')
    oldseqs = seqDB_curr.sequence_set.all()


    from time import time
    t0 = time()
    useFisrt = 1
    failcount = 0

    iterator = SeqIO.parse( INPUT, fmt )
    c = counter(iterator,INF=1,per = 1000)
    iterator = SeqIO.parse( INPUT, fmt )

    with transaction.atomic():
        for obj in iterator:

            acc = header2acc( obj )
            seq = p_clean.sub('',
                              obj._seq.tostring())
            length = len(obj.seq)
            try:
                if acc.islower():
                    raise Exception('accession is all lowercase!')
                jdict = {
                    'acc':acc,
                    'length':length,
                    'seqDB':seqDB_curr,
                }

                seqobjs = oldseqs.filter(**jdict)
                if not seqobjs.exists():
                    seqobj = sequence(**jdict)
                    seqobj.save()
    #                     print "created %s"%jdict['acc']
                elif seqobjs.count() > 1:                   
                    print jdict['acc']
                else:
                    pass

            except Exception as e:
                msg = "%s failed for %s " % (acc, e)
                if debug: print >>sys.__stdout__, msg
                c.fail( msg , acc)
            finally:
                c.count()


    c.summary( behave,INPUT )

    # print "finshed %s from %s" % (behave, INPUT)
    # print '%d instances of %d failed' % ( c.f, c.i)
    # print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
    # failrate = c.f/float(c.i) 
    # assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % (failrate*100)

0 of -1
1act000 failed for accession is all lowercase! 
1afb100 failed for accession is all lowercase! 
1alo006 failed for accession is all lowercase! 
1ar7200 failed for accession is all lowercase! 
1aym100 failed for accession is all lowercase! 
1baa001 failed for accession is all lowercase! 
1bdp001 failed for accession is all lowercase! 
1bdp002 failed for accession is all lowercase! 
1bev100 failed for accession is all lowercase! 
1bev300 failed for accession is all lowercase! 
1bho100 failed for accession is all lowercase! 
1bvp101 failed for accession is all lowercase! 
1bvp102 failed for accession is all lowercase! 
1bvp103 failed for accession is all lowercase! 
1cnt200 failed for accession is all lowercase! 
1cov400 failed for accession is all lowercase! 
1000 of -1
1eah100 failed for accession is all lowercase! 
1eah300 failed for accession is all lowercase! 
1etb200 failed for accession is all lowercase! 
2000 of -1
1fmd100 failed for accession is all lowercase! 
1fmd200 fa

In [16]:
len(obj._seq.tostring())

220

## Importing HMM profiles from .lib file

In [ ]:
%load_ext autoreload
%autoreload 2


In [2]:
header2acc = lambda s:p_cathFAheader.findall(s)[0]
INPUT = full("$SEQlib/cath-S35-hmm3.lib")


In [28]:
'1pkma00'.islower()

True

In [ ]:
seqheader_parse_cath(p_header.findall(hmm_text)[0]).get("acc")

In [7]:
behave = "importing HMM profiles"

from time import time
from tst.domutil.util import *
import re

p_header = re.compile("NAME.*?\n")

lst = parse_hmmlib( INPUT )
c = counter( lst,INF = 1, per = 1000 )
lst = parse_hmmlib( INPUT )
failcount = 0

try:
    with transaction.atomic():
        for hmm_text in lst:
            c.count()
            header = p_header.findall(hmm_text)[0]
            acc = header2acc(header)
            length = int(p_hmmlen.findall(hmm_text)[0])
            
#             acc = next(header_gen)
            try:
                cnode = domain.objects.get(domain_id = acc).classification
                if cnode.hmmprofile_set.exists():
                    pass
                else:
                    hmm = HMMprofile(
                                cath_node_id = cnode.id,
                                text = hmm_text,
                                length = length
                                )
                    hmm.save()
            except Exception as e:
                msg = "failed for %s for Exception:%s" %( acc, e)
                c.fail( msg, acc )
                
except Exception as e:
    print "early stop due to %s" % e


c.summary( behave,INPUT )

# print '%d instances of %d failed' % (c.f, c.i)
# print 'Ended after %.4f sec' % ( time() - t0 )   # len(lst)d
# failrate = c.f / float(c.i) 
# assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate

0 of -1
failed for 1f0cA01 for Exception:get() returned more than one domain -- it returned 2!
1000 of -1
failed for 1imvA01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1imvA02 for Exception:get() returned more than one domain -- it returned 2!
2000 of -1
failed for 1jjoC01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1jmoA01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1jmoA02 for Exception:get() returned more than one domain -- it returned 2!
failed for 1k99A00 for Exception:get() returned more than one domain -- it returned 2!
failed for 1k9oI01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1k9oI02 for Exception:get() returned more than one domain -- it returned 2!
failed for 1l8yA00 for Exception:get() returned more than one domain -- it returned 2!
failed for 1lj5A01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1m

## HMMsearch all the loaded HMM profiles (against CATHS100)

In [ ]:
%load_ext autoreload
%autoreload 0

In [ ]:
from time import time
from tst.domutil.util import *
import re
import gc
# p_header = re.compile("NAME.*?\n")
# from copy import copy
# hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))
seqDB_curr = seqDB.objects.get(name = 'CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
behave = "HMMsearching profiles"

HMMprofile.objects.all()

t0 = time()

qset = HMMprofile.objects.order_by("id")
c = counter(range(qset.count()), per = 100)
batches = batch_qs(qset, batch_size =500)
# it = lst.iterator()
failcount = 0

# print "hi"
def worker():
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/hmmsearch"                
    )

    oldhits = hmm.hit4hmm2hsp_set.all()
    for hit in q_hits:
        hsp = hit[0] ### Assume only one dom per hit
        jdict = hsp2jdict(hsp, query = hmm )
        jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"] ).id

        if not oldhits.filter(**jdict).exists():
            hit_db = hit4hmm2hsp(**jdict)
            hit_db.save()

transaction.set_autocommit( False)
if 1:
# with transaction.atomic():
    for batch in batches:
        for hmm in batch:
            c.count()
#             if c.i > 500:
#                 continue
            try:
                worker()
#                 hits = hmmsearch(hmm,
#                 seqDB_curr = seqDB_curr,
#                 seqDB_file = seqDB_file,
#                 )
            except Exception as e:
                c.fail("failed for %s for %s" % (hmm,e),)
#                 failcount += 1
            if not c.i % c.per:
                transaction.commit()



print "finshed %s from %s" % (behave, hmmlib_file)
print '%d instances of %d failed' % (c.f,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = c.f/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate
transaction.set_autocommit(True)


## Load HMM search results from flatfile (against CATHS100)

In [9]:
from tst.domutil.util import *
from Bio import SearchIO
# seqDB_curr = seqDB.objects.get(name = 'CATH-S35')
seqDB_curr = seqDB.objects.get(name = 'CATH-S40-nr')
# seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"

INPUT = full('$SEQlib/hmm/domtbl/CATHS100.domtbl')
INPUT = full('$SEQlib/hmm/domtbl/CATHS100_v410.domtbl')
INPUT = full('$SEQlib/hmm/domtbl/cath-dataset-nonredundant-S40.domtbl')


#### construct a smaller for testing
# INPUT = split_file(  INPUT ,linecount = 100000)[0]
####

f_handles  = split_file(INPUT,number = 6 )  
parser = SearchIO.parse( INPUT, 'hmmsearch3-domtab' )


c0 = counter([],INF=1,per = 2)

            
acc2hmm_raw  = dict(HMMprofile.objects.defer('text').values_list("cath_node__domain__domain_id","id"))
acc2seq_raw = dict(seqDB_curr.sequence_set.filter().values_list('acc','id'))



In [4]:
qset = hit4hmm2hsp.objects.filter(target__seqDB=seqDB_curr)

In [8]:
# print seqDB_curr
# qset.count()
qset.delete()

(68097L, {u'tst.hit4hmm2hsp': 68097L})

In [41]:
__name__='__main__'
__name__

'__main__'

In [10]:

def parse_worker(fakefile, fmt = 'hmmsearch3-domtab'):
#     print >> sys.__stdout__,fakefile.closed
    parser = SearchIO.parse(fakefile, fmt)
    for q_hits in parser:
    #     print q_hits
        acc = p_cathdomain.findall(q_hits.id)[0]

        try:
            c1 = counter([],INF=-1,ifprint = 0 )
            query_id = acc2hmm[ acc ]
            Jdict = {'query_id':query_id,
                    'hits' : [], }
            for hit in q_hits:
                hsp = hit[0]
                target_acc = p_cathdomain.findall(hit.id)[0]
                try:
                    jdict = hsp2jdict( hsp,simple = 1)
                    jdict['query_id'] = query_id
                    jdict['target_id'] = acc2seq[target_acc]
                    Jdict['hits'].append(jdict)
                except BaseException as e:
                    c1.fail('', target_acc)
                finally:
                    c1.count()

            if c1.f:
                c0.fail('%d of %d instances failed for HMM %s \n Most Recent Exception' % (c1.f, c1.f, acc, c1.e ), acc)
            q.put( Jdict )


        except BaseException as e:
            try:
                c0.fail('%s failed for %s'%(q_hits.id, e), q_hits.id)
            except:
                pass
        finally:
            c0.count()

            
            
def db_listener():
    c = counter([],INF=1, stdout = sys.__stdout__)
    with transaction.atomic():
        while 1:
            
            try:
                obj = q.get()
                c.count()
                if not obj:
                    break
                else:
                    hmm = HMMprofile.objects.defer('text').get(id = obj['query_id'])
                    oldhits = list(
                        hmm.hits.values_list('id',flat = True)
                    )
                    bulk = []

                    for jdict in obj['hits']:
                        if jdict['target_id'] in oldhits:
                            continue
                        else:
                            db_hit = hit4hmm2hsp(**jdict)
                            bulk.append( db_hit )
#                             db_hit.save()
                    hit4hmm2hsp.objects.bulk_create( bulk )
                    
            except BaseException as e:
                print e
                c.fail('%s'%e,)
#                 break
    return c
# tpf=TemporaryFile('w+')


In [11]:
%%time
# f_handles = [f0]
ts = []
# for pcount in [9,10,11,12]:
pcount = 9


# INPUT = full('$SEQlib/hmm/CATHS100_v410.domtbl')
# INPUT = split_file(  INPUT ,linecount = 10000)[0]


import sys
if 1:
    f_handles = split_file( INPUT,number = pcount - 2)
#     f_handles = split_file( f0,number = pcount - 2)


    import os
    import multiprocessing as mp
    # from 
    from multiprocessing.managers import BaseManager, SyncManager
    # class MyManager(BaseManager): pass
    class MyManager(SyncManager): pass
    MyManager.register('counter',counter)
    from time import time

    t0 = time()
    if __name__=='__main__':
    #     m = mp.Manager()
        m = MyManager()
        m.start()
    #     raise Exception
        c0 = m.counter([],INF=1, ifprint = 1)
        acc2hmm = m.dict(acc2hmm_raw)
        acc2seq = m.dict(acc2seq_raw)

        q = m.Queue();          

        if 1:
            pool = mp.Pool( pcount )
    #     with mp.Pool(mp.cpu_count() - 1) as pool:
            watcher  = pool.apply_async(db_listener, args = [])
            jobs = []
            for f_handle in f_handles:
                print f_handle

                job  = pool.apply_async(parse_worker, args = (f_handle,))
                jobs.append(job)

            for job in jobs:
                job.get()
            q.put(None)
            watcher.get()
            
            
        ###### Delete temporary files
        for f in f_handles:
            os.remove(f)


    t = time() - t0
    ts.append(t)
    print pcount, t 
    
    # print time() - t0

0 of -1
/tmp/feng8rcHG7/0
/tmp/feng8rcHG7/1
/tmp/feng8rcHG7/2
/tmp/feng8rcHG7/3
/tmp/feng8rcHG7/4
/tmp/feng8rcHG7/5
/tmp/feng8rcHG7/6
100 of -1
 200 of -1
200 of -1
 200 of -1
 300 of -1
300 of -1
400 of -1
500 of -1
600 of -1
700 of -1
800 of -1
900 of -1
cath|4_1_0|2h4pA02/201-302 failed for '2h4pA02'
1000 of -1
1100 of -1
cath|4_1_0|3o58e00/3-50 failed for '3o58e00'
1200 of -1
1300 of -1
cath|4_1_0|2r9yA01/46-213_311-361 failed for '2r9yA01'
cath|4_1_0|2r9yA02/214-310_362-419 failed for '2r9yA02'
1400 of -1
1500 of -1
1600 of -1
1700 of -1
1800 of -1
1900 of -1
2000 of -1
2100 of -1
2200 of -1
2300 of -1
cath|4_1_0|4if8B01/35-214_310-413 failed for '4if8B01'
2400 of -1
2500 of -1
2600 of -1
2700 of -1
2800 of -1
 2800 of -1
2900 of -1
cath|4_1_0|3ozqA01/14-50_188-292_364-389 failed for '3ozqA01'
cath|4_1_0|3ozqA02/51-187_293-331 failed for '3ozqA02'
3000 of -1
3100 of -1
3200 of -1
3300 of -1
3400 of -1
cath|4_1_0|4afxA02/222-317 failed for '4afxA02'
3500 of -1
 3500 of -1
3600 of -

In [46]:
####Manual removal of temp files
for f in f_handles:
            os.remove(f)

In [ ]:
jdict = {
    'name':'CATH-S40-nr',
    'version':'4_1_0'}
seqDB_curr = verify_exist_entry(jdict,seqDB)

INPUT = full("$SEQlib/cath-dataset-nonredundant-S40.fa")
fmt = 'fasta'
p_cathFAheader=re.compile('\|([0-9,a-z,A-Z]*)\/')
header2acc = lambda s:p_cathFAheader.findall(s.id)[0]
